In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from functools import partial
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import os

In [3]:
def get_dataset(url, file_name):
    
    print("Getting page url")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.set_page_load_timeout(60)
    driver.get(url)
    
    max_click_SHOW_MORE = 10
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, "sc-ftvSup.xPMpo")))
    
    print("Loading more information")

    for i in range(max_click_SHOW_MORE): 
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, "sc-ftvSup.xPMpo")))
        elem = driver.find_elements(By.CLASS_NAME, "sc-ftvSup.xPMpo")
        elem[0].click()
       
    print("Collecting data")
    enderecos = []
    regioes = []
    tipos_valores = []
    outras_informacoes = []
    div = driver.find_element(By.CLASS_NAME,"sc-1y4z098-0.geAIvx")
    time.sleep(2)
    for i in range(30):
        enderecos.append([x.text for x in driver.find_elements(By.CSS_SELECTOR, 'span[data-testid="house-card-address"]')])
        regioes.append([x.text for x in driver.find_elements(By.CSS_SELECTOR, 'span[data-testid="house-card-region"]')])
        tipos_valores.append([x.text for x in  driver.find_elements(By.CLASS_NAME, 'sc-gsnTZi.iRsaMY.sc-crXcEl.jddosl.CozyTypography')])
        outras_informacoes.append(driver.find_elements(By.CSS_SELECTOR, 'small[data-testid="house-card-area"]'))
        driver.execute_script("arguments[0].scrollBy(0, 500);", div)

    outras_informacoes_clean = []
    for i in outras_informacoes:
        for j in i:
            outras_informacoes_clean.append(j.text)

    enderecos = [item for sublist in enderecos for item in sublist]
    regioes = [item for sublist in regioes for item in sublist]
    tipos_valores = [item for sublist in tipos_valores for item in sublist]

    tipo = []
    valores = []

    for i in range(len(tipos_valores)):
        if tipos_valores[i].startswith("R$"):
            valores.append(tipos_valores[i])
        else:
            tipo.append(tipos_valores[i])
    metragem = []
    n_quartos = []
    n_vagas = []
    for info in outras_informacoes_clean:
        splited_info  = info.split('•')
        metragem.append(int(splited_info[0].strip('m² ')))
        n_quartos.append(int(splited_info[1].strip(" quartos ")))
        n_vagas.append(int((splited_info[2].strip(" vagas "))))
    valores = [item.strip("R$ ").replace(".","")  for item in valores]
    regioes = [item.strip("São Paulo").strip(',')  for item in regioes]
    
    print("Building the dataset")

    real_state = {"tipo": tipo, "enderecos" : enderecos, "regioes" :  regioes, "valores" : valores, "metragem" : metragem, "n_quartos" : n_quartos, "n_vagas" : n_vagas}
    df = pd.DataFrame(real_state)
    df.to_csv(file_name, index=False)
    
    print("Done")


In [6]:
file_name= "real_state.csv"
if not os.path.exists(file_name):
    get_dataset('https://www.quintoandar.com.br/comprar/imovel/sao-paulo-sp-brasil', file_name)
df = pd.read_csv(file_name)

In [7]:
df

,tipo,enderecos,regioes,valores,metragem,n_quartos,n_vagas
0,Casa,Rua José de Maio,Interlagos,2600000,450,4,5
1,Apartamento,Rua Justo Azambuja,Cambuci,320000,93,2,0
2,Casa,Rua Maestro Eduardo de Guarnieri,Interlagos,1700000,215,2,2
3,Apartamento,Avenida Alcântara Machado,Brás,190000,70,2,0
4,Studio e kitnet,Rua Fernão Dias,inheiros,420000,28,1,0
...,...,...,...,...,...,...,...
3625,Apartamento,Rua Glicério,Vila Santana,431000,64,2,1
3626,Casa em condomínio,Rua André Natale,Jardim Popular,289000,64,2,2
3627,Apartamento,Rua Barão de Jaguara,Mooca,250000,66,2,0
3628,Apartamento,Rua Doutor Valentim Amaral,Cambuci,355000,34,2,1
